In [107]:
import pandas as pd
import numpy as np
import os
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

In [108]:
clientId = os.environ.get('spotifyClientId')
clientSecret = os.environ.get('spotifyClientSecret')
clientCredentialsManager = SpotifyClientCredentials(client_id=clientId, client_secret=clientSecret)
sp = spotipy.Spotify(client_credentials_manager=clientCredentialsManager) #spotify object to access API

spotifyUsername =os.environ.get("spotifyUsername")
scope = 'user-top-read user-read-recently-played'
SPOTIPY_REDIRECT_URI=os.environ.get('redirect_uri')
token = util.prompt_for_user_token(spotifyUsername, scope, clientId, clientSecret,redirect_uri=SPOTIPY_REDIRECT_URI)
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)


## Get User Data

In [109]:
def get_my_recently_played(lim = 50):
    return sp._get('me/player/recently-played', limit=lim)

def get_my_playlists(lim =50):
    return sp.current_user_playlists(lim)

def get_my_saved_albums(lim=50):
    return sp._get('me/albums', limit=lim)

def get_my_saved_tracks(lim=50):
    return sp._get('me/tracks', limit=lim)

def get_my_followed_artists(lim=50):
    return sp._get('me/following', type='artist', limit=lim)

def get_my_top_artists(lim = 50, t_range = 'long_term'): #t_range is short_term, medium_term, or long_term
    return sp.current_user_top_artists(time_range=t_range, limit=lim)

def get_my_top_tracks(lim = 50, t_range= 'long_term'):
    return sp.current_user_top_tracks(time_range=t_range, limit=lim)

## Get Music Track Data

In [122]:
def get_sample_of_tracks(genre, lim = 50):
    return sp.search('genre:"{}"'.format(genre), type="track", limit=lim)
    
def get_sample_of_artists(genre, lim = 50):
    return sp.search('genre:"{}"'.format(genre), type="track", limit=lim)

def get_audio_analysis(track_id):
    return sp.audio_analysis(track_id)

def get_audio_features(track_id):
    #return sp._get('audio-features/?ids=' + trackid)
    return sp.audio_features(track_id)

def get_search_item(item, of_type):
    #of_type: should be 'artist', 'album', 'track', 'playlist'
    results = sp.search(q= of_type + ':' + item, type=of_type)
    search_items = results[of_type+"s"]['items']
    if len(search_items) > 0:
        return search_items[0]
    else:
        return None
    
def get_artist_albums(artist_id): 
    albums = []
    results = sp.artist_albums(artist_id, album_type='album')
    for album in results['items']:
        name = album['name']
        print(name)
        albums.append(album)
    return albums

def get_album_tracks(album_id):
    tracks = []
    results = sp.album_tracks(album_id)
    for track in results['items']:
        name = track['name']
        print(name)
        tracks.append(track)
    return tracks
    

def get_related_artists(artist_ids): #can use multiple artist ids
    related_artist_ids = set()
    results = sp.recommendations(seed_artists = artist_ids)
    #print(results["tracks"])
    for item in results['tracks']:
        for artist in item["album"]['artists']:
            artist_id =artist['id']
            print(artist['name'])#this allows duplicates, but the ids will not return duplicates
            if artist_id not in artist_ids:
                related_artist_ids.add(artist_id)
    return related_artist_ids

def get_artist_top_tracks(artist_id):
    return sp.artist_top_tracks(artist_id)

def get_new_releases(country = "US", lim=20):
    return sp.new_releases(country, lim)